<a href="https://colab.research.google.com/github/andrewfagence/Google-Gemma-XAI/blob/main/Study_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Dependencies**

Install dependencies via the '!pip install' command, which allows the Colab notebook to access the required libraries for downloading the Google Gemma LLM models, XAI methods and tools, benchmarking tools etc.

In [ ]:
#Install Dependencies
#Install condacolab, setup the conda notebook details, and install and upgrade pip.
!pip install -q condacolab
import condacolab
condacolab.install()
!conda create --name tf
!conda activate tf
!pip install --upgrade pip

#Libraries which the Google Gemma LLM models are dependant on for importing and using
!pip install -q keras-nlp
!pip install -q transformers
!pip install -q tensorflow
!pip install -q accelerate
!pip install -q torch
!pip install -q torchinfo

#Library for benchmarking tools to assess the selected Google Gemma LLM models
!pip install -q deepeval

#Libraries for importing and using XAI tools and methods
!pip install -q lime
!pip install -q bertviz
!pip install -q datasets

#Libraries for improving the selected Google Gemma LLM models
!pip install -q peft

# **Set Environment Variables and Setup Backend Configurations**

Set environment variables and setup backend configurations which allows the necessary libraries required to be imported into the colab notebook, as well as the runtime instance having the correct settings/configurations. They also allow for the access of Google Gemma models on various hosting platforms such as Kaggle and Hugging Face, using their API with the various secrets attached to this colab notebook.

In [ ]:
#Import necessary libraries
import os
from google.colab import userdata
import tensorflow as tf
from tensorflow import keras
import torch
import keras_nlp
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import plot_model, model_to_dot
from datasets import load_dataset
from torchinfo import summary
from transformers import AutoTokenizer, AutoConfig, AutoModel, AutoModelForCausalLM, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from google.colab import drive

#Setup the runtime instance with the correct settings/configurations
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KERAS_BACKEND"] = "tensorflow"  # Or "jax" or "torch".

#Mount Google Drive so figures and diagrams can be saved to a Google Drive Cloud location (For documentation purposes)
drive.mount('/content/drive')

# **Import Selected Google Gemma LLM Models using Keras and the Kaggle API, and Print Out Model Architectures**

## Google Gemma 2B LLM Model

Import a Google Gemma 2B LLM model using the Kaggle API, and print out a summary of its architecture.

In [ ]:
gemma_lm2b = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en") #Import the "gemma_2b_en" model variation being hosted by Keras. Source:https://www.kaggle.com/models/google/gemma/keras/gemma_2b_en
gemma_lm2b.summary() #Display a summary of the Google Gemma 2B LLM model.
plot_model(gemma_lm2b, to_file='/content/drive/My Drive/Gemma_2b_eng_modelplot.png', show_dtype=True, show_layer_names=True, expand_nested=True, show_layer_activations=True, show_trainable=True) #Print out the architecture of the Google Gemma 2B LLM model, and save it to the specified Google Drive location.

## Google Gemma 2B Instruction-Tuned LLM Model

Create a Google Gemma 2B Instruction-Tuned LLM model and print out a summary of its architecture.

In [ ]:
gemma_lm_it2b = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en") #Import the "gemma_instruct_2b_en" model variation being hosted by Keras. Source:https://www.kaggle.com/models/google/gemma/keras/gemma_instruct_2b_en
gemma_lm_it2b.summary() #Display a summary of the Google Gemma 2B Instruction-Tuned LLM model.
plot_model(gemma_lm_it2b, to_file='/content/drive/My Drive/Gemma_2b_instruct_eng_modelplot.png', show_dtype=True, show_layer_names=True, expand_nested=True, show_layer_activations=True, show_trainable=True) #Print out the architecture of the Google Gemma 2B Instruction-Tuned LLM model, and save it to the specified Google Drive location.

# **Gather Benchmark Results/Performance Metrics of the Selected Google Gemma LLM Models Before XAI Implementation**

## Setup the Google Gemma LLM models to be evaluated, this step allows the selected models to function with the DeepEval library

In [ ]:
#Setup the Google Gemma LLM models to be evaluated for benchmarking. Source:https://docs.confident-ai.com/docs/benchmarks-introduction#benchmarking-your-llm
from deepeval.models.base_model import DeepEvalBaseLLM

class Gemma2BEN(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # Load the device onto a CUDA GPU for faster processing

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Gemma 2B EN"

#Create the model and tokenizer for the 'gemma_2b_en' model.
#model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")
#tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")

#Create the model and tokenizer for the 'gemma_instruct_2b_en' model
model2 = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
tokenizer2 = AutoTokenizer.from_pretrained("google/gemma-2b-it")

#Create the models using the deepeval class and defined function
#gemma_2B_EN = Gemma2BEN(model=model, tokenizer=tokenizer)
gemma_2B_EN_IT = Gemma2BEN(model=model2, tokenizer=tokenizer2)

## Import the necessary libraries from the deepeval package

In [ ]:
#Import the necessary benchmark tools and tasks from the deepeval library. Source:https://docs.confident-ai.com/docs/getting-started
from deepeval.benchmarks import MMLU, HellaSwag, BigBenchHard, TruthfulQA, GSM8K #Import the benchmark type
from deepeval.benchmarks.tasks import MMLUTask, HellaSwagTask, BigBenchHardTask, TruthfulQATask #Import the tasks for each benchmark type
from deepeval.benchmarks.modes import TruthfulQAMode #Import the QAmode to be used on TruthfulQA benchmark.

## GSM8K Benchmark

Setup the GSM8K benchmark to be performed on the Google Gemma LLM models.

In [ ]:
#Define benchmark with specific tasks and shots for GSM8K. Source:https://docs.confident-ai.com/docs/benchmarks-gsm8k
#To be used on the 'gemma_2b_en' model.
benchmarkGSM8K = GSM8K(
    n_problems=1319, #1319 is all problems available for the GSM8K benchmark.
    n_shots=1, #Go through all problems only once (1-shot learning).
    enable_cot=False #Disable Chain-of-Thoughts (CoT) prompting.
)

#To be used on the 'gemma_instruct_2b_en' model.
benchmark2GSM8K = GSM8K(
    n_problems=1319, #1319 is all problems available for the GSM8K benchmark.
    n_shots=1, #Go through all problems only once (1-shot learning).
    enable_cot=False #Disable Chain-of-Thoughts (CoT) prompting.
)

Perform the GSM8K benchmark evaluation on the 'gemma_2b_en' model, and print out the score.

In [ ]:
#Perform the GSM8K benchmark on the 'gemma_2b_en' Google Gemma LLM model and print the results
benchmarkGSM8K.evaluate(model=gemma_2B_EN)
print(benchmarkGSM8K.overall_score)

Perform the GSM8K benchmark evaluation on the 'gemma_instruct_2b_en' model, and print out the score.

In [ ]:
#Perform the GSM8K benchmark on the 'gemma_instruct_2b_en' Google Gemma LLM model and print the results
benchmark2GSM8K.evaluate(model=gemma_2B_EN_IT)
print(benchmark2GSM8K.overall_score)

## TruthfulQA Benchmark

Setup the TruthfulQA benchmark to be performed on the Google Gemma LLM models.

In [ ]:
# Define benchmark with specific tasks and modes for TruthfulQA. Source:https://docs.confident-ai.com/docs/benchmarks-truthful-qa
#To be used on the 'gemma_2b_en' model.
benchmarkTruthfulQA = TruthfulQA(
    mode=TruthfulQAMode.MC2 #Perform all 817 tasks under MC2 benchmarking mode
)

#To be used on the 'gemma_instruct_2b_en' model.
benchmark2TruthfulQA = TruthfulQA(
    mode=TruthfulQAMode.MC2 #Perform all 817 tasks under MC2 benchmarking mode
)

Perform the TruthfulQA benchmark evaluation on the 'gemma_2b_en' model, and print out the score.

In [ ]:
#Perform the HumanEval benchmark on the 'gemma_2b_en' Google Gemma LLM model and print the results
benchmarkTruthfulQA.evaluate(model=gemma_2B_EN)
print(benchmarkTruthfulQA.overall_score)

Perform the TruthfulQA benchmark evaluation on the 'gemma_instruct_2b_en' model, and print out the score.

In [ ]:
#Perform the HumanEval benchmark on the 'gemma_instruct_2b_en' Google Gemma LLM model and print the results
benchmark2TruthfulQA.evaluate(model=gemma_2B_EN_IT)
print(benchmark2TruthfulQA.overall_score)

## BigBenchHard (BBH) Benchmark

Setup the BigBenchHard (BBH) benchmark to be performed on the Google Gemma LLM models.

In [ ]:
# Define benchmark with specific tasks and shots for BigBenchHard (BBH). Source:https://docs.confident-ai.com/docs/benchmarks-big-bench-hard
#To be used on the 'gemma_2b_en' model.
benchmarkBBH = BigBenchHard(
    tasks=[BigBenchHardTask.BOOLEAN_EXPRESSIONS, BigBenchHardTask.CAUSAL_JUDGEMENT, BigBenchHardTask.DATE_UNDERSTANDING, BigBenchHardTask.DISAMBIGUATION_QA, BigBenchHardTask.FORMAL_FALLACIES], #Define the tasks to perform for the BBH benchmark on the 'google_2b_en' model.
    n_shots=1, #Go through each question once (1-shot learning).
    enable_cot=False #Disable Chain-of-Thoughts (CoT)
)

#To be used on the 'gemma_instruct_2b_en' model.
benchmark2BBH = BigBenchHard(
    tasks=[BigBenchHardTask.BOOLEAN_EXPRESSIONS, BigBenchHardTask.CAUSAL_JUDGEMENT, BigBenchHardTask.DATE_UNDERSTANDING, BigBenchHardTask.DISAMBIGUATION_QA, BigBenchHardTask.FORMAL_FALLACIES], #Define the tasks to perform for the BBH benchmark on the 'google_instruct_2b_en' model.
    n_shots=1, #Go through each question once (1-shot learning).
    enable_cot=False #Disable Chain-of-Thoughts (CoT)
)

Perform the BBH benchmark evaluation on the 'gemma_2b_en' model, and print out the score.

In [ ]:
#Perform the BBH benchmark on the 'gemma_2b_en' Google Gemma LLM model and print the results
benchmarkBBH.evaluate(model=gemma_2B_EN)
print(benchmarkBBH.overall_score)

Perform the BBH benchmark evaluation on the 'gemma_instruct_2b_en' model, and print out the score.

In [ ]:
#Perform the BBH benchmark on the 'gemma_instruct_2b_en' Google Gemma LLM model and print the results
benchmark2BBH.evaluate(model=gemma_2B_EN_IT)
print(benchmark2BBH.overall_score)

## HellaSwag Benchmark

Setup the HellaSwag benchmark to be performed on the Google Gemma LLM models.

In [ ]:
# Define benchmark with specific tasks and shots for HellaSwag. Source:https://docs.confident-ai.com/docs/benchmarks-hellaswag
#To be used on the 'gemma_2b_en' model.
benchmarkHellaSwag = HellaSwag(
    tasks=[HellaSwagTask.TRIMMING_BRANCHES_OR_HEDGES, HellaSwagTask.WASHING_HANDS, HellaSwagTask.PLAYING_POOL, HellaSwagTask.ZUMBA, HellaSwagTask.CRICKET, HellaSwagTask.BATON_TWIRLING, HellaSwagTask.PHILOSOPHY_AND_RELIGION, HellaSwagTask.GROOMING_DOG, HellaSwagTask.FIXING_THE_ROOF, HellaSwagTask.FIXING_BICYCLE], #Define the tasks to perform for the HellaSwag benchmark on the 'google_2b_en' model.
    n_shots=1 #Go through each question once (1-shot learning).
)

#To be used on the 'gemma_instruct_2b_en' model.
benchmark2HellaSwag = HellaSwag(
    tasks=[HellaSwagTask.TRIMMING_BRANCHES_OR_HEDGES, HellaSwagTask.WASHING_HANDS, HellaSwagTask.PLAYING_POOL, HellaSwagTask.ZUMBA, HellaSwagTask.CRICKET, HellaSwagTask.BATON_TWIRLING, HellaSwagTask.PHILOSOPHY_AND_RELIGION, HellaSwagTask.GROOMING_DOG, HellaSwagTask.FIXING_THE_ROOF, HellaSwagTask.FIXING_BICYCLE], #Define the tasks to perform for the HellaSwag benchmark on the 'google_instruct_2b_en' model.
    n_shots=1 #Go through each question once (1-shot learning).
)

Perform the HellaSwag benchmark evaluation on the 'gemma_2b_en' model, and print out the score.

In [ ]:
#Perform the HellaSwag benchmark on the 'gemma_2b_en' Google Gemma LLM model and print the results
benchmarkHellaSwag.evaluate(model=gemma_2B_EN)
print(benchmarkHellaSwag.overall_score)

Perform the HellaSwag benchmark evaluation on the 'gemma_instruct_2b_en' model, and print out the score.

In [ ]:
#Perform the HellaSwag benchmark on the 'gemma_instruct_2b_en' Google Gemma LLM model and print the results
benchmark2HellaSwag.evaluate(model=gemma_2B_EN_IT)
print(benchmark2HellaSwag.overall_score)

## MMLU Benchmark

Setup the MMLU benchmark to be performed on the Google Gemma LLM models.

In [ ]:
# Define benchmark with specific tasks and shots for Massive Multitask Language Understanding (MMLU). Source:https://docs.confident-ai.com/docs/benchmarks-mmlu
#To be used on the 'gemma_2b_en' model.
benchmarkMMLU = MMLU(
    tasks=[MMLUTask.BUSINESS_ETHICS, MMLUTask.HIGH_SCHOOL_PHYSICS, MMLUTask.HIGH_SCHOOL_WORLD_HISTORY, MMLUTask.HIGH_SCHOOL_MICROECONOMICS, MMLUTask.HIGH_SCHOOL_BIOLOGY, MMLUTask.PHILOSOPHY, MMLUTask.ANATOMY, MMLUTask.COLLEGE_CHEMISTRY, MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ELECTRICAL_ENGINEERING], #Define the tasks to perform for the MMLU benchmark on the 'google_2b_en' model.
    n_shots=1 #Go through each question once (1-shot learning).
)

#To be used on the 'gemma_instruct_2b_en' model.
benchmark2MMLU = MMLU(
    tasks=[MMLUTask.BUSINESS_ETHICS, MMLUTask.HIGH_SCHOOL_PHYSICS, MMLUTask.HIGH_SCHOOL_WORLD_HISTORY, MMLUTask.HIGH_SCHOOL_MICROECONOMICS, MMLUTask.HIGH_SCHOOL_BIOLOGY, MMLUTask.PHILOSOPHY, MMLUTask.ANATOMY, MMLUTask.COLLEGE_CHEMISTRY, MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ELECTRICAL_ENGINEERING], #Define the tasks to perform for the MMLU benchmark on the 'google_instruct_2b_en' model.
    n_shots=1 #Go through each question once (1-shot learning).
)

Perform the MMLU benchmark evaluation on the 'gemma_2b_en' model, and print out the score.

In [ ]:
#Perform the MMLU benchmark evaluation on the 'gemma_2b_en' model, and print out the score.
benchmarkMMLU.evaluate(model=gemma_2B_EN)
print(benchmarkMMLU.overall_score)

Perform the MMLU benchmark evaluation on the 'gemma_instruct_2b_en' model, and print out the score.

In [ ]:
#Perform the MMLU benchmark evaluation on the 'gemma_instruct_2b_en' model, and print out the score.
benchmark2MMLU.evaluate(model=gemma_2B_EN_IT)
print(benchmark2MMLU.overall_score)

# **Apply XAI methods to the Selected Google Gemma Models for Pre-XAI Modification Explanations Using the Hugging Face API.**

## Import selected Google Gemma models using the HuggingFace API, and create a list of 15 questions in various categories.

Creating a list of 15 questions in different categories, the different XAI tools and methods will use these questions to evaluate the selected Google Gemma LLM models. The questions have been spread over 5 different categories and each question has a different level of complexity, this ensures the selected Google Gemma LLM models are evaluated thoroughly.

In [ ]:
#Create the model and tokenizer for the 'gemma_2b_en' model. Comment out when necessary to use/unuse the model (for resource reasons). To be used with the Bertviz attention-visualisation and HotpotQA XAI tools.
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", output_attentions=True, attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=True)
model.config.is_decoder = True

#Create the model and tokenizer for the 'gemma_2b_en' model. Comment out when necessary to use/unuse the model (for resource reasons). To be used with the LIME XAI tool.
#model = AutoModelForSequenceClassification.from_pretrained("google/gemma-2b", num_labels=2)
#tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=True)

#Create the model and tokenizer for the 'gemma_instruct_2b_en' model. Comment out when necessary to use/unuse the model (for resource reasons). To be used with the Bertviz attention-visualisation and HotpotQA XAI tools.
#model2 = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", output_attentions=True, attn_implementation="eager")
#tokenizer2 = AutoTokenizer.from_pretrained("google/gemma-2b-it", use_fast=True)
#model2.config.is_decoder = True

#Create the model and tokenizer for the 'gemma_instruct_2b_en' model. Comment out when necessary to use/unuse the model (for resource reasons). To be used with the LIME XAI tool.
#model2 = AutoModelForSequenceClassification.from_pretrained("google/gemma-2b-it", num_labels=2)
#tokenizer2 = AutoTokenizer.from_pretrained("google/gemma-2b-it", use_fast=True)

#15 questions in total will be asked over general, maths, science, programming/code and paradoxical categories.
questions = [] #Create a new list to hold the text questions
#3 General Questions
questions.append("How many minutes would it take me to walk from the Northumbria University campus located in Newcastle upon Tyne, to the city centre located in Newcastle upon Tyne?") #Question 1
questions.append("To date, who is the most decorated Olympian?") #Question 2
questions.append("Can you list the 'New 7 Wonders of the World'?") #Question 3
#3 Maths Questions
questions.append("What is the value of 'x' in this equation: 18 + 8x = 30") #Question 4
questions.append("What is the limit of 'sin(x)/x' as x approaches 0?") #Question 5
questions.append("What is the value of 'Pi' to 30 decimal places?") #Question 6
#3 Science Questions
questions.append("What happens to a star once it reaches the end of its life?") #Question 7
questions.append("What is the half-life of hydrogen-3 (tritium)?") #Question 8
questions.append("What is 'CRISPR' technology used for?") #Question 9
#3 Programming/Code Questions
questions.append("Create a pseudocode method/function that checks to see if a number is either even or odd.") #Question 10
questions.append("What is the time complexity and space complexity of the merge sort algorithm?") #Question 11
questions.append("What are the key differences between the programming languages 'Python', 'C' and 'Assembly'?") #Question 12
#3 Paradoxical Questions
questions.append("Is the sentence 'This sentence is false.' true or false?") #Question 13
questions.append("If there was a set of all sets, would that set contain itself?") #Question 14
questions.append("What happens when an unstoppable object collides with an immovable object?") #Question 15

## Apply General (Model-Agnostic) XAI methods to the Google Gemma 2B and Google Gemma 2B Instruction-Tuned models

### LIME Explanations for the Google Gemma 2B Model

LIME stands for "**L**ocal **I**nterpretable **M**odel-agnostic **E**xplanations" and this XAI tool can be used to give explanations to all kinds of machine learning models/AI systems, regardless of their complexity and purpose. The LIME XAI tool aims to give understanding to machine learning models/AI systems by interpreting their behaviour within a specific and particular instance, and using simple model structures to approximate values and behaviour.

With textual models and data, LIME identifies which tokens contributes the most within an input to an LLM model by changing and removing different tokens in an input. The "LimeTextExplaier" library will be used, this library functions by applying an exponential kernel on cosine distance, and restricting explanations to words that are present in documents. The LIME XAI tool will be used on the Google Gemma 2B model with the list of 15 questions created for evaluation and investigation.

In [ ]:
#LIME Explanations to be used on the Google Gemma 2B model. Source: https://lime-ml.readthedocs.io/en/latest/lime.html
#Import the LIME "Text Explainer" library to use the LIME XAI tool on the Google Gemma 2B LLM model
from lime.lime_text import LimeTextExplainer

#Define a function which predicts a tensorflow value for each token in an input being fed into the Google Gemma 2B model, to be used with the LIME text explainer.
def gemma2b_model_predictions(question):
  modelInputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True)
  modelOutputs = model(**modelInputs)
  modelLogits = modelOutputs.logits
  modelProbabilities = torch.nn.functional.softmax(modelLogits, dim=-1).detach().numpy()
  return modelProbabilities

#With every question contained in the question array, use the LIME XAI tool/method and output the results
LIMEexplainer = LimeTextExplainer(class_names=['Negative', 'Positive'])
for question in questions:
  explanation = LIMEexplainer.explain_instance(question, gemma2b_model_predictions, labels=[0, 1], num_samples=100)
  explanation.show_in_notebook(text=question)

### LIME Explanations for the Google Gemma 2B Instruction-Tuned Model

LIME stands for "**L**ocal **I**nterpretable **M**odel-agnostic **E**xplanations" and this XAI tool can be used to give explanations to all kinds of machine learning models/AI systems, regardless of their complexity and purpose. The LIME XAI tool aims to give understanding to machine learning models/AI systems by interpreting their behaviour within a specific and particular instance, and using simple model structures to approximate values and behaviour.

With textual models and data, LIME identifies which tokens contributes the most within an input to an LLM model by changing and removing different tokens in an input. The "LimeTextExplaier" library will be used, this library functions by applying an exponential kernel on cosine distance, and restricting explanations to words that are present in documents. The LIME XAI tool will be used on the Google Gemma 2B instruction-tuned model with the list of 15 questions created for evaluation and investigation.

In [ ]:
# LIME Explanations to be used on the Google Gemma 2B instruction-tuned model. Source: https://lime-ml.readthedocs.io/en/latest/lime.html
#Import the necessary libraries to use the LIME XAI tool on the Google Gemma 2B instruction-tuned LLM model
from lime.lime_text import LimeTextExplainer

#Define a function which predicts a tensorflow value for each token in an input being fed into the Google Gemma 2B instruction-tuned model, to be used with the LIME text explainer.
def gemma2bit_model_predictions(question):
  modelInputs2 = tokenizer2(question, return_tensors='pt', padding=True, truncation=True)
  modelOutputs2 = model2(**modelInputs2)
  modelLogits2 = modelOutputs2.logits
  modelProbabilities2 = torch.nn.functional.softmax(modelLogits2, dim=-1).detach().numpy()
  return modelProbabilities2

#With every question contained in the question array, use the LIME XAI tool/method and output the results
LIMEexplainer2 = LimeTextExplainer(class_names=['Negative', 'Positive'])
for question in questions:
  explanation2 = LIMEexplainer2.explain_instance(question, gemma2bit_model_predictions, labels=[0, 1], num_samples=100)
  explanation2.show_in_notebook(text=question)

## Apply Model-Specific XAI methods to the Google Gemma 2B and Google Gemma 2B Instruction-Tuned Models (LLM/Deep Learning)

### Attention-Visualization Explanations for the Google Gemma 2B Model

The BertViz attention-visualisation XAI tool is an open-source and interactive lens that allows users/developers to peek inside the workings of machine learning models and AI systems, specifically deep learning models and neural network models. The BertViz attention-visualisation XAI tool aims to give understanding to deep learning/neural network models by translating numerical data into diagrams which can be interpreted for investigation and analysis.

The BertViz attention-visualisation XAI tool has 3 different flavours/views available to investigate and analyze deep learning/neural network models. There is the 'head view' which allows users/developers to view attention for 1 or more attention heads in the same layer of a model, the 'model view' which allows users/developers to view attention across all layers and heads within a model, and the 'neuron view' which allows users/developers to view attention for individual neurons within a model. The BertViz attention-visualisation XAI tool will be used on the Google Gemma 2B model with the list of 15 questions created, and with the 'model view' for evaluation and investigation.

In [ ]:
#Use the BertViz Attention-Visualisation library on the Google Gemma 2B model. Source: https://pypi.org/project/bertviz/
# Import the "model_view" library from BertViz, to be used on the Google Gemma 2B model
from bertviz import model_view

# For each question in the list of questions, tokenize the question, feed it into the Google Gemma 2B model to obtain outputs, then apply the BertViz "model_view" XAI tool to obtain attention-visualization outputs
for question in questions:
    inputs = tokenizer(question, return_tensors='pt')
    outputs = model(**inputs)
    attention = outputs.attentions
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    model_view(attention, tokens)

### Attention-Visualization Explanations for the Google Gemma 2B Instruction-Tuned Model

The BertViz attention-visualisation XAI tool is an open-source and interactive lens that allows users/developers to peek inside the workings of machine learning models and AI systems, specifically deep learning models and neural network models. The BertViz attention-visualisation XAI tool aims to give understanding to deep learning/neural network models by translating numerical data into diagrams which can be interpreted for investigation and analysis.

The BertViz attention-visualisation XAI tool has 3 different flavours/views available to investigate and analyze deep learning/neural network models. There is the 'head view' which allows users/developers to view attention for 1 or more attention heads in the same layer of a model, the 'model view' which allows users/developers to view attention across all layers and heads within a model, and the 'neuron view' which allows users/developers to view attention for individual neurons within a model. The BertViz attention-visualisation XAI tool will be used on the Google Gemma 2B instruction-tuned model with the list of 15 questions created, and with the 'model view' for evaluation and investigation.

In [ ]:
#Use the BertViz Attention-Visualisation library on the Google Gemma 2B instruction-tuned model. Source: https://pypi.org/project/bertviz/
# Import the "model_view" library from BertViz, to be used on the Google Gemma 2B instruction-tuned model
from bertviz import model_view

# For each question in the list of questions, tokenize the question, feed it into the Google Gemma 2B instruction-tuned model to obtain outputs, then apply the BertViz "model_view" XAI tool to obtain attention-visualization outputs
for question in questions:
    inputs2 = tokenizer2(question, return_tensors='pt')
    outputs2 = model2(**inputs2)
    attention2 = outputs2.attentions
    tokens2 = tokenizer2.convert_ids_to_tokens(inputs2['input_ids'][0])
    model_view(attention2, tokens2)

## Apply XAI methods to Data/Datasets being fed into the Google Gemma 2B and Google Gemma 2B Instruction-Tuned models

### HotpotQA Explanations for the Google Gemma 2B Model

The HotpotQA XAI tool is a question-answering dataset, created by a collective team of NLP researchers at Carnegie Mellon University, Stanford University, and Université de Montréal. The HotpotQA XAI tool question-answering dataset is Wikipedia-based, with 113k question-answer pairs making up its entirety. These question-answer pairs are split up into various partitions, a 'train' partition which contains 90447 pairs, a 'validation' partition which contains 7405 pairs and a 'test' partition which also contains 7405 pairs.

The HotpotQA XAI tool functions by using natural, multi-hop questions, with strong supervision for supporting facts via Wikipedia-based content to enable more explainable question answering systems. With the Google Gemma 2B model, the HotpotQA XAI tool is applied using the first 15 questions from the validation partition of the 'fullwiki' dataset. The generated answer from the Google Gemma 2B model is compared with the ground truth answer contained within the dataset, and an F1 score is calculated from comparing the generated answer with the ground truth answer with each question.

In [ ]:
#Use the HotpotQA library on the Google Gemma 2B model. Source: https://hotpotqa.github.io/?ref=the-batch-deeplearning-ai
# Load the Wikipedia-based HotpotQA library using the 'fullwiki' option
hpQADataset = load_dataset("hotpot_qa", "fullwiki", trust_remote_code=True)

# Use the 'validation' partition of the 'fullwiki' dataset and place it within its own variable
validationDataset = hpQADataset['validation']

# Select the first 15 questions within the validation samples dataset, to use HotpotQA techniques with the Gemma 2B model
validationSamples = validationDataset.select(range(15))

# For each question in the list of 15 selected validation question, print them out for display and inspection purposes
for vs in validationSamples:
  print(vs)

# Define a function which tokenizes inputs and context strings from each validation sample, for the Google Gemma 2B model to process
def tokenizeInput(question, context):
  contextstr = " ".join(context)
  modelInputs = tokenizer.encode_plus(question, contextstr, return_tensors="pt")
  return modelInputs

# Define a function which computes an F1 for each answer the Google Gemma 2B model, it compares an output prediction with a truthful prediction and calculates precision/recall, which can then be used to calculate an F1 score
def computeF1(pred, true):
  predTokens = pred.lower().split()
  trueTokens = true.lower().split()

  commonTokens = set(predTokens) & set(trueTokens)
  if len(commonTokens) == 0:
    return 0.0

  precision = len(commonTokens) / len(predTokens)
  recall = len(commonTokens) / len(trueTokens)
  f1 = 2 * (precision * recall) / (precision + recall)
  return f1

# For each question in the validation dataset, separate each element into their own variable and input the question into the Gemma 2B model. Compare the output prediction of the Google Gemma 2B model with the truthful prediction and output/print performance metrics.
for i, sample in enumerate(validationSamples):
  question = sample['question']
  context = sample['context']
  groundTruth = sample.get('answer')

  if groundTruth is None:
    print(f"Sample {i+1} has no ground truth text. Skipping.")
    continue

  tokenizedInput = tokenizeInput(question, context)
  inputids = tokenizedInput["input_ids"]
  modelOutputs = model.generate(inputids, max_length=150)
  modelAnswer = tokenizer.decode(modelOutputs[0], skip_special_tokens=True)

  exactMatch = modelAnswer.strip().lower() == groundTruth.strip().lower()
  F1Score = computeF1(modelAnswer, groundTruth)

  print(f"Sample {i+1}")
  print(f"Question: {question}")
  print(f"Generated Answer: {modelAnswer}")
  print(f"Ground Truth: {groundTruth}")
  print(f"Exact Match: {exactMatch}")
  print(f"F1 Score: {F1Score}")
  print("----------------------------------------------------------------------------------------")

### HotpotQA Explanations for the Google Gemma 2B Instruction-Tuned Model

The HotpotQA XAI tool is a question-answering dataset, created by a collective team of NLP researchers at Carnegie Mellon University, Stanford University, and Université de Montréal. The HotpotQA XAI tool question-answering dataset is Wikipedia-based, with 113k question-answer pairs making up its entirety. These question-answer pairs are split up into various partitions, a 'train' partition which contains 90447 pairs, a 'validation' partition which contains 7405 pairs and a 'test' partition which also contains 7405 pairs.

The HotpotQA XAI tool functions by using natural, multi-hop questions, with strong supervision for supporting facts via Wikipedia-based content to enable more explainable question answering systems. With the Google Gemma 2B instruction-tuned model, the HotpotQA XAI tool is applied using the first 15 questions from the validation partition of the 'fullwiki' dataset. The generated answer from the Google Gemma 2B instruction-tuned model is compared with the ground truth answer contained within the dataset, and an F1 score is calculated from comparing the generated answer with the ground truth answer with each question.

In [ ]:
#Use the HotpotQA library on the Google Gemma 2B instruction-tuned model. Source: https://hotpotqa.github.io/?ref=the-batch-deeplearning-ai
# Load the Wikipedia-based HotpotQA library using the 'fullwiki' option
hpQADataset2 = load_dataset("hotpot_qa", "fullwiki", trust_remote_code=True)

# Use the 'validation' partition of the 'fullwiki' dataset and place it within its own variable
validationDataset2 = hpQADataset2['validation']

# Select the first 15 questions within the validation samples dataset, to use HotpotQA techniques with the Gemma 2B instruction-tuned model
validationSamples2 = validationDataset2.select(range(15))

# For each question in the list of 15 selected validation question, print them out for display and inspection purposes
for vs in validationSamples2:
  print(vs)

# Define a function which tokenizes inputs and context strings from each validation sample, for the Google Gemma 2B instruction-tuned model to process
def tokenizeInput(question, context):
  contextstr = " ".join(context)
  modelInputs = tokenizer2.encode_plus(question, contextstr, return_tensors="pt")
  return modelInputs

# Define a function which computes an F1 for each answer the Google Gemma 2B instruction-tuned model, it compares an output prediction with a truthful prediction and calculates precision/recall, which can then be used to calculate an F1 score
def computeF1(pred, true):
  predTokens = pred.lower().split()
  trueTokens = true.lower().split()

  commonTokens = set(predTokens) & set(trueTokens)
  if len(commonTokens) == 0:
    return 0.0

  precision = len(commonTokens) / len(predTokens)
  recall = len(commonTokens) / len(trueTokens)
  f1 = 2 * (precision * recall) / (precision + recall)
  return f1

# For each question in the validation dataset, separate each element into their own variable and input the question into the Gemma 2B instruction-tuned model. Compare the output prediction of the Google Gemma 2B model with the truthful prediction and output/print performance metrics.
for i, sample in enumerate(validationSamples2):
  question = sample['question']
  context = sample['context']
  groundTruth = sample.get('answer')

  if groundTruth is None:
    print(f"Sample {i+1} has no ground truth text. Skipping.")
    continue

  tokenizedInput2 = tokenizeInput(question, context)
  inputids2 = tokenizedInput2["input_ids"]
  modelOutputs2 = model2.generate(inputids2, max_length=150)
  modelAnswer2 = tokenizer2.decode(modelOutputs2[0], skip_special_tokens=True)

  exactMatch = modelAnswer2.strip().lower() == groundTruth.strip().lower()
  F1Score = computeF1(modelAnswer2, groundTruth)

  print(f"Sample {i+1}")
  print(f"Question: {question}")
  print(f"Generated Answer: {modelAnswer2}")
  print(f"Ground Truth: {groundTruth}")
  print(f"Exact Match: {exactMatch}")
  print(f"F1 Score: {F1Score}")
  print("----------------------------------------------------------------------------------------")

# **Implement Improvements on the Selected Google Gemma LLM Models with Guidance and Reasoning from XAI Outputs**

## Implement Improvements on the Google Gemma 2B Model

### Import the Google Gemma 2B Model

In [ ]:
#Create the model, tokenizer and configurations for the 'gemma_2b_en' model. To be used within improvement steps.
modelConfig = AutoConfig.from_pretrained("google/gemma-2b")
model = AutoModel.from_pretrained("google/gemma-2b", config=modelConfig)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=True)

### Improvement 3 - Modifying/Tweaking the Architecture of the Google Gemma 2B Model

Print out the configuration settings and model architecture of the Google Gemma 2B model

In [ ]:
#Print out the configuration settings and the architecture of the Google Gemma 2B model for inspection purposes.
print("Model Configurations:")
print(modelConfig)
print("------------------------------------------------------------------------------------")
print("Model Architecture:")
summary(model)

Edit/Tweak the Google Gemma 2B model configuration JSON file with new configurations

In [ ]:
#Create the configurations for the 'gemma_2b_en' model. To be used within improvement steps.
modelConfig = AutoConfig.from_pretrained("google/gemma-2b")

# Edit/Tweak the 'gemma_2b_en' model configuration JSON file with new configurations, print the new model configurations, then set the model to use the newly updated configuration JSON file.
modelConfig.attention_dropout = 0.1 #
modelConfig.torch_dtype = "float32" #
modelConfig.rms_norm_eps = 1e-05 #

modelConfig.save_pretrained("/content/drive/My Drive/ConfigJSONGemma2B/")

modelConfigEdited = AutoConfig.from_pretrained("/content/drive/My Drive/ConfigJSONGemma2B/")

print("Edited/Tweaked Model Configurations:")
print(modelConfigEdited)

model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", config=modelConfigEdited).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=True)

### Improvement 4 - Applying Fine-Tuning Methods/Techniques to the Google Gemma 2B Model

Import and Setup the PEFT library to use LoRA (Low-Rank Adoption)  

In [ ]:
#Import and setup the PEFT library in order to use the LoRA fine-tuning method during training processes. Source: https://huggingface.co/docs/peft/main/en/conceptual_guides/lora
#Import the necessary libraries to use the PEFT fine-tuning tool.
from peft import LoraConfig, get_peft_model, TaskType

#Setup the imported PEFT library for fine-tuning methods during training processes.
loraConfigurations = LoraConfig(
  task_type = TaskType.CAUSAL_LM,
  r = 32,
  lora_alpha = 32,
  lora_dropout = 0.1,
  target_modules = ["q_proj", "v_proj"]
)

#Initialize a PEFT model using the Gemma 2B LLM model and the Lora Configuration variable.
loraModel = get_peft_model(model, loraConfigurations).to("cuda")

### Improvements 1 and 2 - Applying Additional Training & Enable N-Shot Learning for the Google Gemma 2B Model

Wikipedia Dataset

In [ ]:
# Train the Google Gemma 2B model on the Wikipedia dataset. Source: https://huggingface.co/datasets/legacy-datasets/wikipedia
# Retrieve and load a pre-processed version of the 'train' split of the Wikipedia dataset from HuggingFace.
wikipediaDS = load_dataset("wikipedia", "20220301.en", split='train', trust_remote_code=True)

# Randomize the order of the entries within the Wikipedia dataset, and use a seed so it can be reproduced.
wikipediaDS = wikipediaDS.shuffle(seed=42)

# Select 50000 examples from the randomized Wikipedia Dataset (for training time purposes with available resources)
wikipediaDS = wikipediaDS.select(range(50000))

# Split the wikipedia dataset into train and validation sets (90% for train and 10% for validation) for the training processes.
wikipediaDSSplit = wikipediaDS.train_test_split(test_size=0.1)
wikipediaDSTrain = wikipediaDSSplit['train']
wikipediaDSValid = wikipediaDSSplit['test']

# Create a function which will tokenize the 'text' section of a wikipedia dataset entry.
def tokenizeFunctionWP(inputText):
  return tokenizer(inputText['text'], padding=True, truncation=True, max_length=350) # max_length implemented to save environment resources and stop resource errors.

# Tokenize the entire text sections of all the wikipedia train dataset entries, and map this to its own variable to be called upon later.
wikipediaDSTTokenized = wikipediaDSTrain.map(tokenizeFunctionWP, batched=True)

# Tokenize the entire text sections of all the wikipedia validation dataset entries, and map this to its own variable to be called upon later.
wikipediaDSVTokenized = wikipediaDSValid.map(tokenizeFunctionWP, batched=True)

# Initialise the 'TrainingArguments' tool, and input specific arguments for how the model will be trained on the wikipedia dataset.
trainingParamsWP = TrainingArguments(
  output_dir = '/content/drive/My Drive/WikipediaTrainResults',
  eval_strategy = 'epoch',
  per_device_train_batch_size = 2,
  gradient_accumulation_steps = 8,
  num_train_epochs = 5, # Enable the Google Gemma 2B Model for n-shot learning, and allow the model to train on the wikipedia dataset 5 times.
  fp16=True
)

# Use a data collator to batch the wikipedia dataset in preparation for training processes
gemma2BDataColl = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialise the 'Trainer' tool, and input the parameters for the model to be trained on the tokenized wikipedia datasets, as well as the specific training parameter tunings.
modelTrainerWP = Trainer(
  model = loraModel,
  args = trainingParamsWP,
  train_dataset = wikipediaDSTTokenized,
  eval_dataset = wikipediaDSVTokenized,
  data_collator = gemma2BDataColl
)

# Perform the training processes for the Gemma 2B model on the tokenized wikipedia dataset.
modelTrainerWP.train()

### Save the Newly Modified Model (called GemmaPX (Post eXplainability)) to Google Drive to be Called Upon Later for Further Training Stages

In [ ]:
# Save the fine-tuned Google Gemma 2B model and tokenizer to the local directory 'GemmaPX'
loraModel.save_pretrained('/content/drive/My Drive/GemmaPX')
tokenizer.save_pretrained('/content/drive/My Drive/GemmaPX')

### Load the Newly Modified Model (called GemmaPX (Post eXplainability)) from Google Drive to be Called Upon for Further Training Stages

In [ ]:
# Load the Newly Modified Model (called GemmaPX (Post eXplainability)) from Google Drive to be Called Upon for Further Training Stages.
#Create the configurations for the 'gemma_2b_en' model. To be used within improvement steps.
modelConfig = AutoConfig.from_pretrained("google/gemma-2b")

# Edit/Tweak the 'gemma_2b_en' model configuration JSON file with new configurations, print the new model configurations, then set the model to use the newly updated configuration JSON file.
modelConfig.attention_dropout = 0.1 #
modelConfig.torch_dtype = "float32" #
modelConfig.rms_norm_eps = 1e-05 #

modelConfig.save_pretrained("/content/drive/My Drive/ConfigJSONGemma2B/")

modelConfigEdited = AutoConfig.from_pretrained("/content/drive/My Drive/ConfigJSONGemma2B/")

print("Edited/Tweaked Model Configurations:")
print(modelConfigEdited)

# Load the fine-tuned Google Gemma 2B model and tokenizer from the local directory 'GemmaPX'
model = AutoModelForCausalLM.from_pretrained('/content/drive/My Drive/GemmaPX', config=modelConfigEdited).to("cuda")
tokenizer = AutoTokenizer.from_pretrained('/content/drive/My Drive/GemmaPX', use_fast=True)

#Import the necessary libraries to use the PEFT fine-tuning tool.
from peft import LoraConfig, get_peft_model, TaskType

#Setup the imported PEFT library for fine-tuning methods during training processes.
loraConfigurations = LoraConfig(
  task_type = TaskType.CAUSAL_LM,
  r = 32,
  lora_alpha = 32,
  lora_dropout = 0.1,
  target_modules = ["q_proj", "v_proj"]
)

#Initialize a PEFT model using the Gemma 2B LLM model and the Lora Configuration variable.
loraModel = get_peft_model(model, loraConfigurations).to("cuda")

LightEval MATH Dataset

In [ ]:
# Train the Google Gemma 2B model on the LightEval MATH dataset. Source: https://huggingface.co/datasets/lighteval/MATH
# Retrieve and load the LightEval MATH dataset from HuggingFace with the 'all' subset.
mathDS = load_dataset("lighteval/MATH", "all", split='train', trust_remote_code=True)

# Randomize the order of the entries within the LightEval MATH dataset, and use a seed so it can be reproduced.
mathDS = mathDS.shuffle(seed=42)

# Split the LightEval MATH dataset into train and validation sets (90% for train and 10% for validation) for the training processes.
mathDSSplit = mathDS.train_test_split(test_size=0.1)
mathDSTrain = mathDSSplit['train']
mathDSValid = mathDSSplit['test']

# Create a function which will tokenize the 'problem' and 'solution' sections of a LightEval MATH dataset entry.
def tokenizeFunctionMATH(inputText):
  qaString = str(inputText['problem']) + " " + str(inputText['solution'])
  return tokenizer(qaString, padding='max_length', truncation=True, max_length = 350)

# Tokenize the entire text sections of all the LightEval math train dataset entries, and map this to its own variable to be called upon later.
mathDSTTokenized = mathDSTrain.map(tokenizeFunctionMATH)

# Tokenize the entire text sections of all the LightEval validation dataset entries, and map this to its own variable to be called upon later.
mathDSVTokenized = mathDSValid.map(tokenizeFunctionMATH)

# Initialise the 'TrainingArguments' tool, and input specific arguments for how the model will be trained on the MATH dataset.
trainingParamsMATH = TrainingArguments(
  output_dir='/content/drive/My Drive/MATHTrainResults',
  evaluation_strategy='epoch',
  per_device_train_batch_size = 1,
  gradient_accumulation_steps = 1,
  num_train_epochs = 5, # Enable the Google Gemma 2B Model for n-shot learning, and allow the model to train on the math dataset 5 times.
  fp16=True
)

# Use a data collator to batch the LightEval MATH dataset in preparation for training processes
gemma2BDataColl = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialise the 'Trainer' tool, and input the parameters for the model to be trained on the tokenized MATH dataset, as well as the specific training parameter tunings.
modelTrainerMATH = Trainer(
  model = loraModel,
  args = trainingParamsMATH,
  train_dataset = mathDSTTokenized,
  eval_dataset = mathDSVTokenized,
  data_collator = gemma2BDataColl
)

# Perform the training processes for the Gemma 2B model on the tokenized MATH dataset.
modelTrainerMATH.train()

CodeSearchNet Dataset

In [ ]:
# Train the Google Gemma 2B model on the CodeSearchNet dataset. Source: https://huggingface.co/datasets/code-search-net/code_search_net
# Retrieve and load the pre-processed CodeSearchNet dataset from HuggingFace with the 'train' subset.
codesearchnetDS = load_dataset("code-search-net/code_search_net", split="train", trust_remote_code=True)

# Randomize the order of the entries within the CodeSearchNet dataset, and use a seed so it can be reproduced.
codesearchnetDS = codesearchnetDS.shuffle(seed=42)

# Select 50000 examples from the randomized CodeSearchNet Dataset (for training time purposes with available resources)
codesearchnetDS = codesearchnetDS.select(range(50000))

# Split the CodeSearchNet dataset into train and validation sets (90% for train and 10% for validation) for the training processes.
codesearchnetDSSplit = codesearchnetDS.train_test_split(test_size=0.1)
codesearchnetDSTrain = codesearchnetDSSplit['train']
codesearchnetDSValid = codesearchnetDSSplit['test']

# Create a function which will tokenize the 'whole_func_string' section of a CodeSearchNet dataset entry.
def tokenizeFunctionCSN(inputText):
  return tokenizer(inputText['whole_func_string'], padding=True, truncation=True, max_length=350) # max_length implemented to save environment resources and stop resource errors.

# Tokenize the entire text sections of all the CodeSearchNet train dataset entries, and map this to its own variable to be called upon later.
codesearchnetDSTTokenized = codesearchnetDSTrain.map(tokenizeFunctionCSN, batched=True)

# Tokenize the entire text sections of all the CodeSearchNet validation dataset entries, and map this to its own variable to be called upon later.
codesearchnetDSVTokenized = codesearchnetDSValid.map(tokenizeFunctionCSN, batched=True)

# Initialise the 'TrainingArguments' tool, and input specific arguments for how the model will be trained on the CodeSearchNet dataset.
trainingParamsCSN = TrainingArguments(
  output_dir='/content/drive/My Drive/CodeSearchNetTrainResults',
  evaluation_strategy='epoch',
  per_device_train_batch_size = 2,
  gradient_accumulation_steps = 8,
  num_train_epochs = 5, # Enable the Google Gemma 2B Model for n-shot learning, and allow the model to train on the CodeSearchNet dataset 5 times.
  fp16=True
)

# Use a data collator to batch the CodeSearchNet dataset in preparation for training processes
gemma2BDataColl = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialise the 'Trainer' tool, and input the parameters for the model to be trained on the tokenized CodeSearchNet dataset, as well as the specific training parameter tunings.
modelTrainerCSN = Trainer(
  model = loraModel,
  args = trainingParamsCSN,
  train_dataset = codesearchnetDSTTokenized,
  eval_dataset = codesearchnetDSVTokenized,
  data_collator = gemma2BDataColl
)

# Perform the training processes for the Gemma 2B model on the tokenized CodeSearchNet dataset.
modelTrainerCSN.train()

## Implement Improvements on the Google Gemma 2B Instruction-Tuned Model

### Import the Google Gemma 2B Instruction-Tuned Model

In [ ]:
#Create the model, tokenizer and configurations for the 'gemma_instruct_2b_en' model. To be used within improvement steps.
modelConfig2 = AutoConfig.from_pretrained("google/gemma-2b-it")
model2 = AutoModel.from_pretrained("google/gemma-2b-it", config=modelConfig2)
tokenizer2 = AutoTokenizer.from_pretrained("google/gemma-2b-it", use_fast=True)

### Improvement 3 - Modifying/Tweaking the Architecture of the Google Gemma 2B Instruction-Tuned Model

Print out the configuration settings and model architecture of the Google Gemma instruction-tuned 2B model

In [ ]:
#Print out the configuration settings and the architecture of the Google Gemma 2B instruction-tuned model for inspection purposes.
print("Model Configurations:")
print(modelConfig2)
print("------------------------------------------------------------------------------------")
print("Model Architecture:")
summary(model2)

Edit/Tweak the Google Gemma 2B instruction-tuned model configuration JSON file with new configurations

In [ ]:
#Create the configurations for the 'gemma_instruct_2b_en' model. To be used within improvement steps.
modelConfig2 = AutoConfig.from_pretrained("google/gemma-2b-it")

# Edit/Tweak the 'gemma_instruct_2b_en' model configuration JSON file with new configurations, print the new model configurations, then set the model to use the newly updated configuration JSON file.
modelConfig2.attention_dropout = 0.1 #
modelConfig2.torch_dtype = "float32" #
modelConfig2.rms_norm_eps = 1e-05 #

modelConfig2.save_pretrained("/content/drive/My Drive/ConfigJSONGemmaIT2B/")

modelConfigEdited2 = AutoConfig.from_pretrained("/content/drive/My Drive/ConfigJSONGemmaIT2B/")

print("Edited/Tweaked Model Configurations:")
print(modelConfigEdited2)

model2 = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", config=modelConfigEdited2).to("cuda")
tokenizer2 = AutoTokenizer.from_pretrained("google/gemma-2b-it", use_fast=True)

### Improvement 4 - Applying Fine-Tuning Methods/Techniques to the Google Gemma 2B Instruction-Tuned Model

Import and Setup the PEFT library to use LoRA (Low-Rank Adoption)  

In [ ]:
#Import and setup the PEFT library in order to use the LoRA fine-tuning method during training processes. Source: https://huggingface.co/docs/peft/main/en/conceptual_guides/lora
#Import the necessary libraries to use the PEFT fine-tuning tool.
from peft import LoraConfig, get_peft_model, TaskType

#Setup the imported PEFT library for fine-tuning methods during training processes.
loraConfigurations2 = LoraConfig(
  task_type = TaskType.CAUSAL_LM,
  r = 32,
  lora_alpha = 32,
  lora_dropout = 0.1,
  target_modules = ["q_proj", "v_proj"]
)

#Initialize a PEFT model using the Gemma 2B Instruction-Tuned LLM model and the Lora Configuration variable.
loraModel2 = get_peft_model(model2, loraConfigurations2).to("cuda")

### Improvements 1 and 2 - Applying Additional Training & Enable N-Shot Learning for the Google Gemma 2B Instruction-Tuned Model

Wikipedia Dataset

In [ ]:
# Train the Google Gemma 2B instruction-tuned model on the Wikipedia dataset. Source: https://huggingface.co/datasets/legacy-datasets/wikipedia
# Retrieve and load a pre-processed version of the 'train' split of the Wikipedia dataset from HuggingFace.
wikipediaDS2 = load_dataset("wikipedia", "20220301.en", split='train', trust_remote_code=True)

# Randomize the order of the entries within the Wikipedia dataset, and use a seed so it can be reproduced.
wikipediaDS2 = wikipediaDS2.shuffle(seed=42)

# Select 50000 examples from the randomized Wikipedia Dataset (for training time purposes with available resources)
wikipediaDS2 = wikipediaDS2.select(range(50000))

# Split the wikipedia dataset into train and validation sets (90% for train and 10% for validation) for the training processes.
wikipediaDSSplit2 = wikipediaDS2.train_test_split(test_size=0.1)
wikipediaDSTrain2 = wikipediaDSSplit2['train']
wikipediaDSValid2 = wikipediaDSSplit2['test']

# Create a function which will tokenize the 'text' section of a wikipedia dataset entry.
def tokenizeFunctionWP2(inputText):
  return tokenizer2(inputText['text'], padding=True, truncation=True, max_length=350) # max_length implemented to save environment resources and stop resource errors.

# Tokenize the entire text sections of all the wikipedia train dataset entries, and map this to its own variable to be called upon later.
wikipediaDSTTokenized2 = wikipediaDSTrain2.map(tokenizeFunctionWP2, batched=True)

# Tokenize the entire text sections of all the wikipedia validation dataset entries, and map this to its own variable to be called upon later.
wikipediaDSVTokenized2 = wikipediaDSValid2.map(tokenizeFunctionWP2, batched=True)

# Initialise the 'TrainingArguments' tool, and input specific arguments for how the model will be trained on the wikipedia dataset.
trainingParamsWP2 = TrainingArguments(
  output_dir = '/content/drive/My Drive/WikipediaTrainResults2',
  eval_strategy = 'epoch',
  per_device_train_batch_size = 2,
  gradient_accumulation_steps = 8,
  num_train_epochs = 5, # Enable the Google Gemma 2B instruction-tuned Model for n-shot learning, and allow the model to train on the wikipedia dataset 5 times.
  fp16=True
)

# Use a data collator to batch the wikipedia dataset in preparation for training processes
gemma2BDataColl2 = DataCollatorForLanguageModeling(tokenizer=tokenizer2, mlm=False)

# Initialise the 'Trainer' tool, and input the parameters for the model to be trained on the tokenized wikipedia datasets, as well as the specific training parameter tunings.
modelTrainerWP2 = Trainer(
  model = loraModel2,
  args = trainingParamsWP2,
  train_dataset = wikipediaDSTTokenized2,
  eval_dataset = wikipediaDSVTokenized2,
  data_collator = gemma2BDataColl2
)

# Perform the training processes for the Gemma 2B instruction-tuned model on the tokenized wikipedia dataset.
modelTrainerWP2.train()

### Save the Newly Modified Model (called GemmaITPX (Post eXplainability)) to Google Drive to be Called Upon Later for Further Training Stages

In [ ]:
# Save the fine-tuned Google Gemma 2B Instruction-Tuned model and tokenizer to the local directory 'GemmaITPX'
loraModel2.save_pretrained('/content/drive/My Drive/GemmaITPX')
tokenizer2.save_pretrained('/content/drive/My Drive/GemmaITPX')

### Load the Newly Modified Model (called GemmaITPX (Post eXplainability)) from Google Drive to be Called Upon for Further Training Stages

In [ ]:
# Load the Newly Modified Model (called GemmaITPX (Post eXplainability)) from Google Drive to be Called Upon for Further Training Stages.
#Create the configurations for the 'gemma_instruct_2b_en' model. To be used within improvement steps.
modelConfig2 = AutoConfig.from_pretrained("google/gemma-2b-it")

# Edit/Tweak the 'gemma_instruct_2b_en' model configuration JSON file with new configurations, print the new model configurations, then set the model to use the newly updated configuration JSON file.
modelConfig2.attention_dropout = 0.1 #
modelConfig2.torch_dtype = "float32" #
modelConfig2.rms_norm_eps = 1e-05 #

modelConfig2.save_pretrained("/content/drive/My Drive/ConfigJSONGemmaIT2B/")

modelConfigEdited2 = AutoConfig.from_pretrained("/content/drive/My Drive/ConfigJSONGemmaIT2B/")

print("Edited/Tweaked Model Configurations:")
print(modelConfigEdited2)

# Load the fine-tuned Google Gemma 2B instruction-tuned model and tokenizer from the local directory 'GemmaITPX'
model2 = AutoModelForCausalLM.from_pretrained('/content/drive/My Drive/GemmaITPX', config=modelConfigEdited2).to("cuda")
tokenizer2 = AutoTokenizer.from_pretrained('/content/drive/My Drive/GemmaITPX', use_fast=True)

#Import the necessary libraries to use the PEFT fine-tuning tool.
from peft import LoraConfig, get_peft_model, TaskType

#Setup the imported PEFT library for fine-tuning methods during training processes.
loraConfigurations2 = LoraConfig(
  task_type = TaskType.CAUSAL_LM,
  r = 32,
  lora_alpha = 32,
  lora_dropout = 0.1,
  target_modules = ["q_proj", "v_proj"]
)

#Initialize a PEFT model using the Gemma 2B Instruction-Tuned LLM model and the Lora Configuration variable.
loraModel2 = get_peft_model(model2, loraConfigurations2).to("cuda")

LightEval MATH Dataset

In [ ]:
# Train the Google Gemma 2B instruction-tuned model on the LightEval MATH dataset. Source: https://huggingface.co/datasets/lighteval/MATH
# Retrieve and load the LightEval MATH dataset from HuggingFace with the 'all' subset.
mathDS2 = load_dataset("lighteval/MATH", "all", split='train', trust_remote_code=True)

# Randomize the order of the entries within the LightEval MATH dataset, and use a seed so it can be reproduced.
mathDS2 = mathDS2.shuffle(seed=42)

# Split the LightEval MATH dataset into train and validation sets (90% for train and 10% for validation) for the training processes.
mathDSSplit2 = mathDS2.train_test_split(test_size=0.1)
mathDSTrain2 = mathDSSplit2['train']
mathDSValid2 = mathDSSplit2['test']

# Create a function which will tokenize the 'problem' and 'solution' sections of a LightEval MATH dataset entry.
def tokenizeFunctionMATH2(inputText):
  qaString2 = str(inputText['problem']) + " " + str(inputText['solution'])
  return tokenizer2(qaString2, padding='max_length', truncation=True, max_length=350)

# Tokenize the entire text sections of all the LightEval math train dataset entries, and map this to its own variable to be called upon later.
mathDSTTokenized2 = mathDSTrain2.map(tokenizeFunctionMATH2)

# Tokenize the entire text sections of all the LightEval validation dataset entries, and map this to its own variable to be called upon later.
mathDSVTokenized2 = mathDSValid2.map(tokenizeFunctionMATH2)

# Initialise the 'TrainingArguments' tool, and input specific arguments for how the model will be trained on the MATH dataset.
trainingParamsMATH2 = TrainingArguments(
  output_dir='/content/drive/My Drive/MATHTrainResults2',
  evaluation_strategy='epoch',
  per_device_train_batch_size = 1,
  gradient_accumulation_steps = 1,
  num_train_epochs = 5, # Enable the Google Gemma 2B instruction-tuned Model for n-shot learning, and allow the model to train on the math dataset 5 times.
  fp16=True
)

# Use a data collator to batch the LightEval MATH dataset in preparation for training processes
gemma2BDataColl2 = DataCollatorForLanguageModeling(tokenizer=tokenizer2, mlm=False)

# Initialise the 'Trainer' tool, and input the parameters for the model to be trained on the tokenized MATH dataset, as well as the specific training parameter tunings.
modelTrainerMATH2 = Trainer(
  model = loraModel2,
  args = trainingParamsMATH2,
  train_dataset = mathDSTTokenized2,
  eval_dataset = mathDSVTokenized2,
  data_collator = gemma2BDataColl2
)

# Perform the training processes for the Gemma 2B instruction-tuned model on the tokenized MATH dataset.
modelTrainerMATH2.train()

CodeSearchNet Dataset

In [ ]:
# Train the Google Gemma 2B instruction-tuned model on the CodeSearchNet dataset. Source: https://huggingface.co/datasets/code-search-net/code_search_net
# Retrieve and load the pre-processed CodeSearchNet dataset from HuggingFace with the 'train' subset.
codesearchnetDS2 = load_dataset("code-search-net/code_search_net", split="train", trust_remote_code=True)

# Randomize the order of the entries within the CodeSearchNet dataset, and use a seed so it can be reproduced.
codesearchnetDS2 = codesearchnetDS2.shuffle(seed=42)

# Select 50000 examples from the randomized CodeSearchNet Dataset (for training time purposes with available resources)
codesearchnetDS2 = codesearchnetDS2.select(range(50000))

# Split the CodeSearchNet dataset into train and validation sets (90% for train and 10% for validation) for the training processes.
codesearchnetDSSplit2 = codesearchnetDS2.train_test_split(test_size=0.1)
codesearchnetDSTrain2 = codesearchnetDSSplit2['train']
codesearchnetDSValid2 = codesearchnetDSSplit2['test']

# Create a function which will tokenize the 'whole_func_string' section of a CodeSearchNet dataset entry.
def tokenizeFunctionCSN2(inputText):
  return tokenizer2(inputText['whole_func_string'], padding=True, truncation=True, max_length=350) # max_length implemented to save environment resources and stop resource errors.

# Tokenize the entire text sections of all the CodeSearchNet train dataset entries, and map this to its own variable to be called upon later.
codesearchnetDSTTokenized2 = codesearchnetDSTrain2.map(tokenizeFunctionCSN2, batched=True)

# Tokenize the entire text sections of all the CodeSearchNet validation dataset entries, and map this to its own variable to be called upon later.
codesearchnetDSVTokenized2 = codesearchnetDSValid2.map(tokenizeFunctionCSN2, batched=True)

# Initialise the 'TrainingArguments' tool, and input specific arguments for how the model will be trained on the CodeSearchNet dataset.
trainingParamsCSN2 = TrainingArguments(
  output_dir='/content/drive/My Drive/CodeSearchNetTrainResults2',
  evaluation_strategy='epoch',
  per_device_train_batch_size = 2,
  gradient_accumulation_steps = 8,
  num_train_epochs = 5, # Enable the Google Gemma 2B instruction-tuned Model for n-shot learning, and allow the model to train on the CodeSearchNet dataset 5 times.
  fp16=True
)

# Use a data collator to batch the CodeSearchNet dataset in preparation for training processes
gemma2BDataColl2 = DataCollatorForLanguageModeling(tokenizer=tokenizer2, mlm=False)

# Initialise the 'Trainer' tool, and input the parameters for the model to be trained on the tokenized CodeSearchNet dataset, as well as the specific training parameter tunings.
modelTrainerCSN2 = Trainer(
  model = loraModel2,
  args = trainingParamsCSN2,
  train_dataset = codesearchnetDSTTokenized2,
  eval_dataset = codesearchnetDSVTokenized2,
  data_collator = gemma2BDataColl2
)

# Perform the training processes for the Gemma 2B model on the tokenized CodeSearchNet dataset.
modelTrainerCSN2.train()

# **Gather Benchmark Results/Performance Metrics of the Selected Google Gemma LLM Models After XAI Implementation**

## Setup the Google Gemma LLM models to be evaluated, this step allows the modified models to function with the DeepEval library

In [ ]:
#Setup the modified Google Gemma LLM models to be evaluated for benchmarking. Source:https://docs.confident-ai.com/docs/benchmarks-introduction#benchmarking-your-llm
from deepeval.models.base_model import DeepEvalBaseLLM

class Gemma2BENPX(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # Load the device onto a CUDA GPU for faster processing

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Gemma 2B EN PX"

#Create the model and tokenizer for the 'Gemma PX' model.
#model = AutoModelForCausalLM.from_pretrained("/content/drive/My Drive/GemmaPX")
#tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/GemmaPX")

#Create the model and tokenizer for the 'Gemma IT PX' modified model
model2 = AutoModelForCausalLM.from_pretrained("/content/drive/My Drive/GemmaITPX")
tokenizer2 = AutoTokenizer.from_pretrained("/content/drive/My Drive/GemmaITPX")

#Create the models using the deepeval class and defined function
#gemma_PX = Gemma2BENPX(model=model, tokenizer=tokenizer)
gemma_IT_PX = Gemma2BENPX(model=model2, tokenizer=tokenizer2)

## Import the necessary libraries from the deepeval package

In [ ]:
#Import the necessary benchmark tools and tasks from the deepeval library. Source:https://docs.confident-ai.com/docs/getting-started
from deepeval.benchmarks import MMLU, HellaSwag, BigBenchHard, TruthfulQA, GSM8K #Import the benchmark type
from deepeval.benchmarks.tasks import MMLUTask, HellaSwagTask, BigBenchHardTask, TruthfulQATask #Import the tasks for each benchmark type
from deepeval.benchmarks.modes import TruthfulQAMode #Import the QAmode to be used on TruthfulQA benchmark.

## GSM8K Benchmark

Setup the GSM8K benchmark to be performed on the modified Google Gemma LLM models.

In [ ]:
#Define benchmark with specific tasks and shots for GSM8K. Source:https://docs.confident-ai.com/docs/benchmarks-gsm8k
#To be used on the 'Gemma PX' model.
benchmarkGSM8K = GSM8K(
    n_problems=1319, #1319 is all problems available for the GSM8K benchmark.
    n_shots=1, #Go through all problems only once (1-shot learning).
    enable_cot=False #Disable Chain-of-Thoughts (CoT) prompting.
)

#To be used on the 'Gemma IT PX' model.
benchmark2GSM8K = GSM8K(
    n_problems=1319, #1319 is all problems available for the GSM8K benchmark.
    n_shots=1, #Go through all problems only once (1-shot learning).
    enable_cot=False #Disable Chain-of-Thoughts (CoT) prompting.
)

Perform the GSM8K benchmark evaluation on the 'Gemma PX' model, and print out the score.

In [ ]:
#Perform the GSM8K benchmark on the modified 'Gemma PX' Google Gemma LLM model and print the results
benchmarkGSM8K.evaluate(model=gemma_PX)
print(benchmarkGSM8K.overall_score)

Perform the GSM8K benchmark evaluation on the 'Gemma IT PX' model, and print out the score.

In [ ]:
#Perform the GSM8K benchmark on the modified 'Gemma IT PX' Google Gemma LLM model and print the results
benchmark2GSM8K.evaluate(model=gemma_IT_PX)
print(benchmark2GSM8K.overall_score)

## TruthfulQA Benchmark

Setup the TruthfulQA benchmark to be performed on the modified Google Gemma LLM models.

In [ ]:
# Define benchmark with specific tasks and modes for TruthfulQA. Source:https://docs.confident-ai.com/docs/benchmarks-truthful-qa
#To be used on the 'Gemma PX' model.
benchmarkTruthfulQA = TruthfulQA(
    mode=TruthfulQAMode.MC2 #Perform all 817 tasks under MC2 benchmarking mode
)

#To be used on the 'Gemma IT PX' model.
benchmark2TruthfulQA = TruthfulQA(
    mode=TruthfulQAMode.MC2 #Perform all 817 tasks under MC2 benchmarking mode
)

Perform the TruthfulQA benchmark evaluation on the modified 'Gemma PX' model, and print out the score.

In [ ]:
#Perform the HumanEval benchmark on the modified 'Gemma PX' Google Gemma LLM model and print the results
benchmarkTruthfulQA.evaluate(model=gemma_PX)
print(benchmarkTruthfulQA.overall_score)

Perform the TruthfulQA benchmark evaluation on the modified 'Gemma IT PX' model, and print out the score.

In [ ]:
#Perform the HumanEval benchmark on the modified 'Gemma IT PX' Google Gemma LLM model and print the results
benchmark2TruthfulQA.evaluate(model=gemma_IT_PX)
print(benchmark2TruthfulQA.overall_score)

## BigBenchHard (BBH) Benchmark

Setup the BigBenchHard (BBH) benchmark to be performed on the modified Google Gemma LLM models.

In [ ]:
# Define benchmark with specific tasks and shots for BigBenchHard (BBH). Source:https://docs.confident-ai.com/docs/benchmarks-big-bench-hard
#To be used on the 'Gemma PX' model.
benchmarkBBH = BigBenchHard(
    tasks=[BigBenchHardTask.BOOLEAN_EXPRESSIONS, BigBenchHardTask.CAUSAL_JUDGEMENT, BigBenchHardTask.DATE_UNDERSTANDING, BigBenchHardTask.DISAMBIGUATION_QA, BigBenchHardTask.FORMAL_FALLACIES], #Define the tasks to perform for the BBH benchmark on the 'Gemma PX' model.
    n_shots=1, #Go through each question once (1-shot learning).
    enable_cot=False #Disable Chain-of-Thoughts (CoT)
)

#To be used on the 'Gemma IT PX' model.
benchmark2BBH = BigBenchHard(
    tasks=[BigBenchHardTask.BOOLEAN_EXPRESSIONS, BigBenchHardTask.CAUSAL_JUDGEMENT, BigBenchHardTask.DATE_UNDERSTANDING, BigBenchHardTask.DISAMBIGUATION_QA, BigBenchHardTask.FORMAL_FALLACIES], #Define the tasks to perform for the BBH benchmark on the 'Gemma IT PX' model.
    n_shots=1, #Go through each question once (1-shot learning).
    enable_cot=False #Disable Chain-of-Thoughts (CoT)
)

Perform the BBH benchmark evaluation on the 'Gemma PX' model, and print out the score.

In [ ]:
#Perform the BBH benchmark on the 'Gemma PX' Google Gemma LLM model and print the results
benchmarkBBH.evaluate(model=gemma_PX)
print(benchmarkBBH.overall_score)

Perform the BBH benchmark evaluation on the 'Gemma IT PX' model, and print out the score.

In [ ]:
#Perform the BBH benchmark on the 'Gemma IT PX' Google Gemma LLM model and print the results
benchmark2BBH.evaluate(model=gemma_IT_PX)
print(benchmark2BBH.overall_score)

## HellaSwag Benchmark

Setup the HellaSwag benchmark to be performed on the modified Google Gemma LLM models.

In [ ]:
# Define benchmark with specific tasks and shots for HellaSwag. Source:https://docs.confident-ai.com/docs/benchmarks-hellaswag
#To be used on the 'Gemma PX' model.
benchmarkHellaSwag = HellaSwag(
    tasks=[HellaSwagTask.TRIMMING_BRANCHES_OR_HEDGES, HellaSwagTask.WASHING_HANDS, HellaSwagTask.PLAYING_POOL, HellaSwagTask.ZUMBA, HellaSwagTask.CRICKET, HellaSwagTask.BATON_TWIRLING, HellaSwagTask.PHILOSOPHY_AND_RELIGION, HellaSwagTask.GROOMING_DOG, HellaSwagTask.FIXING_THE_ROOF, HellaSwagTask.FIXING_BICYCLE], #Define the tasks to perform for the HellaSwag benchmark on the 'Gemma PX' model.
    n_shots=1 #Go through each question once (1-shot learning).
)

#To be used on the 'Gemma IT PX' model.
benchmark2HellaSwag = HellaSwag(
    tasks=[HellaSwagTask.TRIMMING_BRANCHES_OR_HEDGES, HellaSwagTask.WASHING_HANDS, HellaSwagTask.PLAYING_POOL, HellaSwagTask.ZUMBA, HellaSwagTask.CRICKET, HellaSwagTask.BATON_TWIRLING, HellaSwagTask.PHILOSOPHY_AND_RELIGION, HellaSwagTask.GROOMING_DOG, HellaSwagTask.FIXING_THE_ROOF, HellaSwagTask.FIXING_BICYCLE], #Define the tasks to perform for the HellaSwag benchmark on the 'Gemma IT PX' model.
    n_shots=1 #Go through each question once (1-shot learning).
)

Perform the HellaSwag benchmark evaluation on the 'Gemma PX' model, and print out the score.

In [ ]:
#Perform the HellaSwag benchmark on the 'Gemma PX' Google Gemma LLM model and print the results
benchmarkHellaSwag.evaluate(model=gemma_PX)
print(benchmarkHellaSwag.overall_score)

Perform the HellaSwag benchmark evaluation on the 'Gemma IT PX' model, and print out the score.

In [ ]:
#Perform the HellaSwag benchmark on the 'Gemma IT PX' Google Gemma LLM model and print the results
benchmark2HellaSwag.evaluate(model=gemma_IT_PX)
print(benchmark2HellaSwag.overall_score)

## MMLU Benchmark

Setup the MMLU benchmark to be performed on the modified Google Gemma LLM models.

In [ ]:
# Define benchmark with specific tasks and shots for Massive Multitask Language Understanding (MMLU). Source:https://docs.confident-ai.com/docs/benchmarks-mmlu
#To be used on the 'Gemma PX' model.
benchmarkMMLU = MMLU(
    tasks=[MMLUTask.BUSINESS_ETHICS, MMLUTask.HIGH_SCHOOL_PHYSICS, MMLUTask.HIGH_SCHOOL_WORLD_HISTORY, MMLUTask.HIGH_SCHOOL_MICROECONOMICS, MMLUTask.HIGH_SCHOOL_BIOLOGY, MMLUTask.PHILOSOPHY, MMLUTask.ANATOMY, MMLUTask.COLLEGE_CHEMISTRY, MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ELECTRICAL_ENGINEERING], #Define the tasks to perform for the MMLU benchmark on the 'Gemma PX' model.
    n_shots=1 #Go through each question once (1-shot learning).
)

#To be used on the 'gemma_instruct_2b_en' model.
benchmark2MMLU = MMLU(
    tasks=[MMLUTask.BUSINESS_ETHICS, MMLUTask.HIGH_SCHOOL_PHYSICS, MMLUTask.HIGH_SCHOOL_WORLD_HISTORY, MMLUTask.HIGH_SCHOOL_MICROECONOMICS, MMLUTask.HIGH_SCHOOL_BIOLOGY, MMLUTask.PHILOSOPHY, MMLUTask.ANATOMY, MMLUTask.COLLEGE_CHEMISTRY, MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ELECTRICAL_ENGINEERING], #Define the tasks to perform for the MMLU benchmark on the 'Gemma IT PX' model.
    n_shots=1 #Go through each question once (1-shot learning).
)

Perform the MMLU benchmark evaluation on the modified 'Gemma PX' model, and print out the score.

In [ ]:
#Perform the MMLU benchmark evaluation on the 'Gemma PX' model, and print out the score.
benchmarkMMLU.evaluate(model=gemma_PX)
print(benchmarkMMLU.overall_score)

Perform the MMLU benchmark evaluation on the modified 'Gemma IT PX' model, and print out the score.

In [ ]:
#Perform the MMLU benchmark evaluation on the 'Gemma IT PX' model, and print out the score.
benchmark2MMLU.evaluate(model=gemma_IT_PX)
print(benchmark2MMLU.overall_score)

# **Apply XAI methods to the Selected Google Gemma Models for Post-XAI Modification Explanations Using the Hugging Face API.**

## Import modified Google Gemma models using Google Drive, and create a list of 15 questions in various categories.

Creating a list of 15 questions in different categories, the different XAI tools and methods will use these questions to evaluate the modified Google Gemma LLM models. The questions have been spread over 5 different categories and each question has a different level of complexity, this ensures the modified Google Gemma LLM models are evaluated thoroughly.

In [ ]:
#Create the model and tokenizer for the 'Gemma PX' model. Comment out when necessary to use/unuse the model (for resource reasons). To be used with the Bertviz attention-visualisation and HotpotQA XAI tools.
#model = AutoModelForCausalLM.from_pretrained("/content/drive/My Drive/GemmaPX", output_attentions=True, attn_implementation="eager")
#tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/GemmaPX", use_fast=True)
#model.config.is_decoder = True

#Create the model and tokenizer for the 'Gemma PX' model. Comment out when necessary to use/unuse the model (for resource reasons). To be used with the LIME XAI tool.
#model = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/GemmaPX", num_labels=2)
#tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/GemmaPX", use_fast=True)

#Create the model and tokenizer for the 'Gemma IT PX' model. Comment out when necessary to use/unuse the model (for resource reasons). To be used with the Bertviz attention-visualisation and HotpotQA XAI tools.
#model2 = AutoModelForCausalLM.from_pretrained("/content/drive/My Drive/GemmaITPX", output_attentions=True, attn_implementation="eager")
#tokenizer2 = AutoTokenizer.from_pretrained("/content/drive/My Drive/GemmaITPX", use_fast=True)
#model2.config.is_decoder = True

#Create the model and tokenizer for the 'Gemma IT PX' model. Comment out when necessary to use/unuse the model (for resource reasons). To be used with the LIME XAI tool.
model2 = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/GemmaITPX", num_labels=2)
tokenizer2 = AutoTokenizer.from_pretrained("/content/drive/My Drive/GemmaITPX", use_fast=True)

#15 questions in total will be asked over general, maths, science, programming/code and paradoxical categories.
questions = [] #Create a new list to hold the text questions
#3 General Questions
questions.append("How many minutes would it take me to walk from the Northumbria University campus located in Newcastle upon Tyne, to the city centre located in Newcastle upon Tyne?") #Question 1
questions.append("To date, who is the most decorated Olympian?") #Question 2
questions.append("Can you list the 'New 7 Wonders of the World'?") #Question 3
#3 Maths Questions
questions.append("What is the value of 'x' in this equation: 18 + 8x = 30") #Question 4
questions.append("What is the limit of 'sin(x)/x' as x approaches 0?") #Question 5
questions.append("What is the value of 'Pi' to 30 decimal places?") #Question 6
#3 Science Questions
questions.append("What happens to a star once it reaches the end of its life?") #Question 7
questions.append("What is the half-life of hydrogen-3 (tritium)?") #Question 8
questions.append("What is 'CRISPR' technology used for?") #Question 9
#3 Programming/Code Questions
questions.append("Create a pseudocode method/function that checks to see if a number is either even or odd.") #Question 10
questions.append("What is the time complexity and space complexity of the merge sort algorithm?") #Question 11
questions.append("What are the key differences between the programming languages 'Python', 'C' and 'Assembly'?") #Question 12
#3 Paradoxical Questions
questions.append("Is the sentence 'This sentence is false.' true or false?") #Question 13
questions.append("If there was a set of all sets, would that set contain itself?") #Question 14
questions.append("What happens when an unstoppable object collides with an immovable object?") #Question 15

## Apply General (Model-Agnostic) XAI methods to the Gemma PX and Gemma IT PX Instruction-Tuned models

### LIME Explanations for the Gemma PX Model

LIME stands for "Local Interpretable Model-agnostic Explanations" and this XAI tool can be used to give explanations to all kinds of machine learning models/AI systems, regardless of their complexity and purpose. The LIME XAI tool aims to give understanding to machine learning models/AI systems by interpreting their behaviour within a specific and particular instance, and using simple model structures to approximate values and behaviour.

With textual models and data, LIME identifies which tokens contributes the most within an input to an LLM model by changing and removing different tokens in an input. The "LimeTextExplaier" library will be used, this library functions by applying an exponential kernel on cosine distance, and restricting explanations to words that are present in documents. The LIME XAI tool will be used on the Gemma PX model with the list of 15 questions created for evaluation and investigation.

In [ ]:
#LIME Explanations to be used on the Gemma PX model. Source: https://lime-ml.readthedocs.io/en/latest/lime.html
#Import the LIME "Text Explainer" library to use the LIME XAI tool on the Gemma PX LLM model
from lime.lime_text import LimeTextExplainer

#Define a function which predicts a tensorflow value for each token in an input being fed into the Gemma PX model, to be used with the LIME text explainer.
def gemmapx_model_predictions(question):
  modelInputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True)
  modelOutputs = model(**modelInputs)
  modelLogits = modelOutputs.logits
  modelProbabilities = torch.nn.functional.softmax(modelLogits, dim=-1).detach().numpy()
  return modelProbabilities

#With every question contained in the question array, use the LIME XAI tool/method and output the results
LIMEexplainer = LimeTextExplainer(class_names=['Negative', 'Positive'])
for question in questions:
  explanation = LIMEexplainer.explain_instance(question, gemmapx_model_predictions, labels=[0, 1], num_samples=100)
  explanation.show_in_notebook(text=question)

### LIME Explanations for the Gemma IT PX Model

LIME stands for "**L**ocal **I**nterpretable **M**odel-agnostic **E**xplanations" and this XAI tool can be used to give explanations to all kinds of machine learning models/AI systems, regardless of their complexity and purpose. The LIME XAI tool aims to give understanding to machine learning models/AI systems by interpreting their behaviour within a specific and particular instance, and using simple model structures to approximate values and behaviour.

With textual models and data, LIME identifies which tokens contributes the most within an input to an LLM model by changing and removing different tokens in an input. The "LimeTextExplaier" library will be used, this library functions by applying an exponential kernel on cosine distance, and restricting explanations to words that are present in documents. The LIME XAI tool will be used on the Gemma IT PX model with the list of 15 questions created for evaluation and investigation.

In [ ]:
# LIME Explanations to be used on the Gemma IT PX model. Source: https://lime-ml.readthedocs.io/en/latest/lime.html
#Import the necessary libraries to use the LIME XAI tool on the Gemma IT PX LLM model
from lime.lime_text import LimeTextExplainer

#Define a function which predicts a tensorflow value for each token in an input being fed into the Gemma IT PX model, to be used with the LIME text explainer.
def gemmaitpx_model_predictions(question):
  modelInputs2 = tokenizer2(question, return_tensors='pt', padding=True, truncation=True)
  modelOutputs2 = model2(**modelInputs2)
  modelLogits2 = modelOutputs2.logits
  modelProbabilities2 = torch.nn.functional.softmax(modelLogits2, dim=-1).detach().numpy()
  return modelProbabilities2

#With every question contained in the question array, use the LIME XAI tool/method and output the results
LIMEexplainer2 = LimeTextExplainer(class_names=['Negative', 'Positive'])
for question in questions:
  explanation2 = LIMEexplainer2.explain_instance(question, gemmaitpx_model_predictions, labels=[0, 1], num_samples=100)
  explanation2.show_in_notebook(text=question)

## Apply Model-Specific XAI methods to the Gemma PX and Gemma IT PX Models (LLM/Deep Learning)

### Attention-Visualization Explanations for the Gemma PX Model

The BertViz attention-visualisation XAI tool is an open-source and interactive lens that allows users/developers to peek inside the workings of machine learning models and AI systems, specifically deep learning models and neural network models. The BertViz attention-visualisation XAI tool aims to give understanding to deep learning/neural network models by translating numerical data into diagrams which can be interpreted for investigation and analysis.

The BertViz attention-visualisation XAI tool has 3 different flavours/views available to investigate and analyze deep learning/neural network models. There is the 'head view' which allows users/developers to view attention for 1 or more attention heads in the same layer of a model, the 'model view' which allows users/developers to view attention across all layers and heads within a model, and the 'neuron view' which allows users/developers to view attention for individual neurons within a model. The BertViz attention-visualisation XAI tool will be used on the Gemma PX model with the list of 15 questions created, and with the 'model view' for evaluation and investigation.

In [ ]:
#Use the BertViz Attention-Visualisation library on the Gemma PX model. Source: https://pypi.org/project/bertviz/
# Import the "model_view" library from BertViz, to be used on the Gemma PX model
from bertviz import model_view

# For each question in the list of questions, tokenize the question, feed it into the Gemma PX model to obtain outputs, then apply the BertViz "model_view" XAI tool to obtain attention-visualization outputs
for question in questions:
    inputs = tokenizer(question, return_tensors='pt')
    outputs = model(**inputs)
    attention = outputs.attentions
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    model_view(attention, tokens)

### Attention-Visualization Explanations for the Gemma IT PX Model

The BertViz attention-visualisation XAI tool is an open-source and interactive lens that allows users/developers to peek inside the workings of machine learning models and AI systems, specifically deep learning models and neural network models. The BertViz attention-visualisation XAI tool aims to give understanding to deep learning/neural network models by translating numerical data into diagrams which can be interpreted for investigation and analysis.

The BertViz attention-visualisation XAI tool has 3 different flavours/views available to investigate and analyze deep learning/neural network models. There is the 'head view' which allows users/developers to view attention for 1 or more attention heads in the same layer of a model, the 'model view' which allows users/developers to view attention across all layers and heads within a model, and the 'neuron view' which allows users/developers to view attention for individual neurons within a model. The BertViz attention-visualisation XAI tool will be used on the Gemma IT PX model with the list of 15 questions created, and with the 'model view' for evaluation and investigation.

In [ ]:
#Use the BertViz Attention-Visualisation library on the Gemma IT PX model. Source: https://pypi.org/project/bertviz/
# Import the "model_view" library from BertViz, to be used on the Gemma IT PX model
from bertviz import model_view

# For each question in the list of questions, tokenize the question, feed it into the Gemma IT PX model to obtain outputs, then apply the BertViz "model_view" XAI tool to obtain attention-visualization outputs
for question in questions:
    inputs2 = tokenizer2(question, return_tensors='pt')
    outputs2 = model2(**inputs2)
    attention2 = outputs2.attentions
    tokens2 = tokenizer2.convert_ids_to_tokens(inputs2['input_ids'][0])
    model_view(attention2, tokens2)

## Apply XAI methods to Data/Datasets being fed into the Gemma PX and Gemma IT PX models

### HotpotQA Explanations for the Gemma PX Model

The HotpotQA XAI tool is a question-answering dataset, created by a collective team of NLP researchers at Carnegie Mellon University, Stanford University, and Université de Montréal. The HotpotQA XAI tool question-answering dataset is Wikipedia-based, with 113k question-answer pairs making up its entirety. These question-answer pairs are split up into various partitions, a 'train' partition which contains 90447 pairs, a 'validation' partition which contains 7405 pairs and a 'test' partition which also contains 7405 pairs.

The HotpotQA XAI tool functions by using natural, multi-hop questions, with strong supervision for supporting facts via Wikipedia-based content to enable more explainable question answering systems. With the Gemma PX model, the HotpotQA XAI tool is applied using the first 15 questions from the validation partition of the 'fullwiki' dataset. The generated answer from the Gemma PX model is compared with the ground truth answer contained within the dataset, and an F1 score is calculated from comparing the generated answer with the ground truth answer with each question.

In [ ]:
#Use the HotpotQA library on the Gemma PX model. Source: https://hotpotqa.github.io/?ref=the-batch-deeplearning-ai
# Load the Wikipedia-based HotpotQA library using the 'fullwiki' option
hpQADataset = load_dataset("hotpot_qa", "fullwiki", trust_remote_code=True)

# Use the 'validation' partition of the 'fullwiki' dataset and place it within its own variable
validationDataset = hpQADataset['validation']

# Select the first 15 questions within the validation samples dataset, to use HotpotQA techniques with the Gemma PX model
validationSamples = validationDataset.select(range(15))

# For each question in the list of 15 selected validation question, print them out for display and inspection purposes
for vs in validationSamples:
  print(vs)

# Define a function which tokenizes inputs and context strings from each validation sample, for the Gemma PX model to process
def tokenizeInput(question, context):
  contextstr = " ".join(context)
  modelInputs = tokenizer.encode_plus(question, contextstr, return_tensors="pt")
  return modelInputs

# Define a function which computes an F1 for each answer the Gemma PX model, it compares an output prediction with a truthful prediction and calculates precision/recall, which can then be used to calculate an F1 score
def computeF1(pred, true):
  predTokens = pred.lower().split()
  trueTokens = true.lower().split()

  commonTokens = set(predTokens) & set(trueTokens)
  if len(commonTokens) == 0:
    return 0.0

  precision = len(commonTokens) / len(predTokens)
  recall = len(commonTokens) / len(trueTokens)
  f1 = 2 * (precision * recall) / (precision + recall)
  return f1

# For each question in the validation dataset, separate each element into their own variable and input the question into the Gemma PX model. Compare the output prediction of the Gemma PX model with the truthful prediction and output/print performance metrics.
for i, sample in enumerate(validationSamples):
  question = sample['question']
  context = sample['context']
  groundTruth = sample.get('answer')

  if groundTruth is None:
    print(f"Sample {i+1} has no ground truth text. Skipping.")
    continue

  tokenizedInput = tokenizeInput(question, context)
  inputids = tokenizedInput["input_ids"]
  modelOutputs = model.generate(inputids, max_length=150)
  modelAnswer = tokenizer.decode(modelOutputs[0], skip_special_tokens=True)

  exactMatch = modelAnswer.strip().lower() == groundTruth.strip().lower()
  F1Score = computeF1(modelAnswer, groundTruth)

  print(f"Sample {i+1}")
  print(f"Question: {question}")
  print(f"Generated Answer: {modelAnswer}")
  print(f"Ground Truth: {groundTruth}")
  print(f"Exact Match: {exactMatch}")
  print(f"F1 Score: {F1Score}")
  print("----------------------------------------------------------------------------------------")

### HotpotQA Explanations for the Gemma IT PX Model

The HotpotQA XAI tool is a question-answering dataset, created by a collective team of NLP researchers at Carnegie Mellon University, Stanford University, and Université de Montréal. The HotpotQA XAI tool question-answering dataset is Wikipedia-based, with 113k question-answer pairs making up its entirety. These question-answer pairs are split up into various partitions, a 'train' partition which contains 90447 pairs, a 'validation' partition which contains 7405 pairs and a 'test' partition which also contains 7405 pairs.

The HotpotQA XAI tool functions by using natural, multi-hop questions, with strong supervision for supporting facts via Wikipedia-based content to enable more explainable question answering systems. With the Gemma IT PX model, the HotpotQA XAI tool is applied using the first 15 questions from the validation partition of the 'fullwiki' dataset. The generated answer from the Gemma IT PX model is compared with the ground truth answer contained within the dataset, and an F1 score is calculated from comparing the generated answer with the ground truth answer with each question.

In [ ]:
#Use the HotpotQA library on the Gemma IT PX model. Source: https://hotpotqa.github.io/?ref=the-batch-deeplearning-ai
# Load the Wikipedia-based HotpotQA library using the 'fullwiki' option
hpQADataset2 = load_dataset("hotpot_qa", "fullwiki", trust_remote_code=True)

# Use the 'validation' partition of the 'fullwiki' dataset and place it within its own variable
validationDataset2 = hpQADataset2['validation']

# Select the first 15 questions within the validation samples dataset, to use HotpotQA techniques with the Gemma IT PX model
validationSamples2 = validationDataset2.select(range(15))

# For each question in the list of 15 selected validation question, print them out for display and inspection purposes
for vs in validationSamples2:
  print(vs)

# Define a function which tokenizes inputs and context strings from each validation sample, for the Gemma IT PX instruction-tuned model to process
def tokenizeInput(question, context):
  contextstr = " ".join(context)
  modelInputs = tokenizer2.encode_plus(question, contextstr, return_tensors="pt")
  return modelInputs

# Define a function which computes an F1 for each answer the Google Gemma 2B instruction-tuned model, it compares an output prediction with a truthful prediction and calculates precision/recall, which can then be used to calculate an F1 score
def computeF1(pred, true):
  predTokens = pred.lower().split()
  trueTokens = true.lower().split()

  commonTokens = set(predTokens) & set(trueTokens)
  if len(commonTokens) == 0:
    return 0.0

  precision = len(commonTokens) / len(predTokens)
  recall = len(commonTokens) / len(trueTokens)
  f1 = 2 * (precision * recall) / (precision + recall)
  return f1

# For each question in the validation dataset, separate each element into their own variable and input the question into the Gemma 2B instruction-tuned model. Compare the output prediction of the Google Gemma 2B model with the truthful prediction and output/print performance metrics.
for i, sample in enumerate(validationSamples2):
  question = sample['question']
  context = sample['context']
  groundTruth = sample.get('answer')

  if groundTruth is None:
    print(f"Sample {i+1} has no ground truth text. Skipping.")
    continue

  tokenizedInput2 = tokenizeInput(question, context)
  inputids2 = tokenizedInput2["input_ids"]
  modelOutputs2 = model2.generate(inputids2, max_length=150)
  modelAnswer2 = tokenizer2.decode(modelOutputs2[0], skip_special_tokens=True)

  exactMatch = modelAnswer2.strip().lower() == groundTruth.strip().lower()
  F1Score = computeF1(modelAnswer2, groundTruth)

  print(f"Sample {i+1}")
  print(f"Question: {question}")
  print(f"Generated Answer: {modelAnswer2}")
  print(f"Ground Truth: {groundTruth}")
  print(f"Exact Match: {exactMatch}")
  print(f"F1 Score: {F1Score}")
  print("----------------------------------------------------------------------------------------")